In [61]:
# Import Block
import numpy as np
from rdkit import Chem
from openbabel import pybel
from openff.toolkit.topology import Molecule,Topology
from openff.toolkit.utils import RDKitToolkitWrapper
import openmm
from simtk.openmm import app
from openmm import unit
from openmm.app import PDBFile
from openff.toolkit.typing.engines.smirnoff import ForceField
from openmm.vec3 import Vec3
from scipy.optimize import minimize,Bounds
import warnings
warnings.simplefilter("ignore")


In [2]:
# Wrapper and FF setup
# Use RDKit wrapper
rdktkw = RDKitToolkitWrapper()

# Loading setup parameters
forcefield = ForceField('openff-2.0.0.offxml')

In [3]:
# load pdb with one copy of pdb file
#off_mol = Molecule.from_pdb_and_smiles('7101899.pdb', "CC1=CN=C(C(=C1OC)C)C[S@@](=O)C2=NC3=C(N2)C=C(C=C3)OC")
off_mol = Molecule.from_pdb_and_smiles('1100249.pdb', "O=C1N([C@H]2C=C[C@@H]1[C@H]1C[C@@H]2C=C1)C(=O)O[C@H]1[C@@H](CC[C@H](C1)C)C(C)C")



In [4]:
# load supercell pdb file (2x2x2) into topology
pdb_file = PDBFile('1100249_supercell.pdb')
off_top = Topology.from_openmm(pdb_file.topology, [off_mol])
# Create MD simulation inputs
system = forcefield.create_openmm_system(off_top)
integrator = openmm.VerletIntegrator(1*unit.femtoseconds)
platform = openmm.Platform.getPlatformByName('Reference')

/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [6]:
# create simulation
simulation = openmm.app.Simulation(pdb_file.topology, system, integrator, platform)

In [7]:
# set initial positions from pdbfile
positions = pdb_file.getPositions()
simulation.context.setPositions(positions)

In [8]:
# set reporters
pdb_reporter = openmm.app.PDBReporter('trajectory.pdb', 1)
state_data_reporter = openmm.app.StateDataReporter(
    "data.csv",
    1,
    step=True,
    potentialEnergy=True,
    temperature=True,
    density=True,
)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(state_data_reporter)

In [9]:
simulation.context.setPositions(positions)
simulation.saveState('initial')
orig_potential = simulation.context.getState(getEnergy=True).getPotentialEnergy()
#state_data_reporter.report(simulation, simulation.context.getState())
print('Initial Energy ' + str(orig_potential))
print('Minimizing Energy!')
simulation.minimizeEnergy()
min_state = simulation.context.getState(getEnergy=True, getPositions=True, getForces=True)
min_potential = min_state.getPotentialEnergy()
print('Final Energy = ' + str(min_potential))


Initial Energy 11890.525741791453 kJ/mol
Minimizing Energy!
Final Energy = 7801.333976616224 kJ/mol


In [15]:
# try running a 0 step
simulation.step(1)


In [16]:
import mdtraj

initial = mdtraj.load_pdb('1100249_supercell.pdb')
final = mdtraj.load_pdb('trajectory.pdb')
rmsd = mdtraj.rmsd(initial,final)
print(rmsd[0])

0.014685853


In [107]:
# Need block to feed parameters to minimizer, where forces provide 3*n derivatives of energy
# wrt position, +3 more derivatives of energy wrt box vectors
# This block is a work in progress

def box_energy(x,*args):
    # x is np array of 3*n(atoms) positional coordinates and 6 coordinates of 3 triclinical box vectors
    # Return the energy of the system (in kJ/mol)
    # *args will have the simulation context and n (number of particles)
    context = args[0]
    n = args[1]
    # Build position array
    positions_arr = np.empty([n,3])
    for i in range(int(n)):
        positions_arr[i][0] = x[i*3]
        positions_arr[i][1] = x[i*3+1]
        positions_arr[i][2] = x[i*3+2]
    # Build periodic box vectors
    a = np.array([x[n*3],0,0])
    b = np.array([x[n*3+1],x[n*3+2],0])
    c = np.array([x[n*3+3],x[n*3+4],x[n*3+5]])
    # Set Context with positions and periodic boundary conditions
    context.setPositions(positions_arr)
    #print(positions_arr)
    context.setPeriodicBoxVectors(a,b,c)
    # Return Energy
    energy = context.getState(getEnergy=True).getPotentialEnergy().value_in_unit(unit.kilojoule_per_mole)
    return energy

def jacobian(x,*args):
    #must return a n*3 + 6 size vector with derivative of energy with respect to each input parameter
    #for positions, return given forces
    context = args[0]
    n = args[1]

    energy = context.getState(getEnergy=True).getPotentialEnergy().value_in_unit(unit.kilojoule_per_mole)
    forces = -1*context.getState(getForces=True).getForces(asNumpy=True).value_in_unit(unit.kilojoule_per_mole/(unit.nano*unit.meter))
    jac = forces.flatten()
    # Positions
    positions_arr = np.empty([n,3])
    for i in range(int(n)):
        positions_arr[i][:] = x[i*3:i*3+3]
    context.setPositions(positions_arr)
    # box vectors
    a = np.array([x[n*3],0,0])
    b = np.array([x[n*3+1],x[n*3+2],0])
    c = np.array([x[n*3+3],x[n*3+4],x[n*3+5]])
    epsilon = 1e-6
    box_vectors = np.array([a,b,c])
    frac_positions = np.matmul(np.linalg.inv(box_vectors),positions_arr.T).T

    # a_x
    temp_a_x = x[n*3] + epsilon
    temp_box_vectors = np.array([np.array([temp_a_x,0,0]),b,c]) # Replace appropriate entry in box vectors
    new_positions = np.matmul(temp_box_vectors,frac_positions.T).T # Transform frac coordiantes with new box vectors
    temp_x = np.concatenate((new_positions.flatten(),[temp_a_x],x[n*3+1:n*3+6])) # build x array to feed to box_energy function with new box vector
    new_energy = box_energy(temp_x,context,n) #compute new energy
    grad_temp = (new_energy-energy)/epsilon # compute gradient
    jac = np.append(jac,grad_temp) # append to jac

    # b_x
    temp_b_x = x[n*3+1] + epsilon
    temp_box_vectors = np.array([a,np.array([temp_b_x,x[n*3+2],0]),c])
    new_positions = np.matmul(temp_box_vectors,frac_positions.T).T
    temp_x = np.concatenate((new_positions.flatten(),[x[n*3]],[temp_b_x],x[n*3+2:n*3+6]))
    new_energy = box_energy(temp_x,context,n)
    grad_temp = (new_energy-energy)/epsilon
    jac = np.append(jac,grad_temp)

    # b_y
    temp_b_y = x[n*3+2] + epsilon
    temp_box_vectors = np.array([a,np.array([x[n*3+1],temp_b_y,0]),c])
    new_positions = np.matmul(temp_box_vectors,frac_positions.T).T
    temp_x = np.concatenate((new_positions.flatten(),x[n*3:n*3+2],[temp_b_y],x[n*3+3:n*3+6]))
    new_energy = box_energy(temp_x,context,n)
    grad_temp = (new_energy-energy)/epsilon
    jac = np.append(jac,grad_temp)

    #c_x
    temp_c_x = x[n*3+3] + epsilon
    temp_box_vectors = np.array([a,b,np.array([temp_c_x,x[n*3+4],x[n*3+5]])])
    new_positions = np.matmul(temp_box_vectors,frac_positions.T).T
    temp_x = np.concatenate((new_positions.flatten(),x[n*3:n*3+3],[temp_c_x],x[n*3+4:n*3+6]))
    new_energy = box_energy(temp_x,context,n)
    grad_temp = (new_energy-energy)/epsilon
    jac = np.append(jac,grad_temp)

    #c_y
    temp_c_y = x[n*3+4] + epsilon
    temp_box_vectors = np.array([a,b,np.array([x[n*3+3],temp_c_y,x[n*3+5]])])
    new_positions = np.matmul(temp_box_vectors,frac_positions.T).T
    temp_x = np.concatenate((new_positions.flatten(),x[n*3:n*3+4],[temp_c_y],[x[n*3+5]]))
    new_energy = box_energy(temp_x,context,n)
    grad_temp = (new_energy-energy)/epsilon
    jac = np.append(jac,grad_temp)

    #c_z
    temp_c_z = x[n*3+5] + epsilon
    temp_box_vectors = np.array([a,b,np.array([x[n*3+3],x[n*3+4],temp_c_z])])
    new_positions = np.matmul(temp_box_vectors,frac_positions.T).T
    temp_x = np.concatenate((new_positions.flatten(),x[n*3:n*3+5],[temp_c_z]))
    new_energy = box_energy(temp_x,context,n)
    grad_temp = (new_energy-energy)/epsilon
    jac = np.append(jac,grad_temp)

    return jac

def constraint_1(x,*args):
    n=args[1]
    return x[n*3] - 2*x[n*3+1]
def constraint_2(x,*args):
    n=args[1]
    return x[n*3] - 2*x[n*3+3]
def constraint_3(x,*args):
    n=args[1]
    return x[n*3+2] - 2*x[n*3+4]

In [101]:
# Dummy variables to test
test_state = simulation.context.getState(getPositions=True)
test_positions = test_state.getPositions(asNumpy=True)
pdb_file.topology.getUnitCellDimensions()
box_vectors = pdb_file.topology.getPeriodicBoxVectors()
A= np.array(box_vectors.value_in_unit(unit.nano*unit.meter))
x_test = test_positions.flatten()
x_test = np.append(x_test,[A[0][0],A[1][0],A[1][1],A[2][0],A[2][1],A[2][2]])
n_test = len(test_positions)
test = jacobian(x_test,simulation.context,n_test)
print(test)
print(test[n_test*3])

[-3.20986505e+01  3.82252077e+00 -2.58769107e+01 ...  6.91562543e+10
 -2.17366842e+10  1.54618968e+10]
12764246536.254883


In [110]:
lb = -100*np.ones(n_test*3+6)
lb[n_test*3:n_test*3+6] = [1.8,1.8,1.8,1.8,1.8,1.8]
bounds = Bounds(lb=lb,ub=np.inf,keep_feasible=True)
result = minimize(box_energy,x_test,(simulation.context,n_test),method='L-BFGS-B',jac=jacobian,bounds=bounds,constraints=[{'type':'ineq','fun':constraint_1},{'type':'ineq','fun':constraint_2},{'type':'ineq','fun':constraint_3}],options={'maxiter': 1000})
print(result)
print(result.x-x_test)


OpenMMException: Periodic box vectors must be in reduced form.

In [95]:
# Example for tranformation of cartesian coordinates to fractional
pdb_file.topology.getUnitCellDimensions()
box_vectors = pdb_file.topology.getPeriodicBoxVectors()
A= np.array(box_vectors.value_in_unit(unit.nano*unit.meter))
print(A)
A_inv = np.linalg.inv(A)
x = np.array([[1.2,1.2,1.2],[1.5,1.5,1.5]])
y = np.matmul(A_inv,x.T).T
A[1][1] = A[1][1]*1.1
y_modified = np.matmul(A,y.T).T
print(result.x[n_test*3:n_test*3+6])
print(result.x[n_test*3:n_test*3+6]-x_test[n_test*3:n_test*3+6])

[[2.1786 0.     0.    ]
 [0.     2.5623 0.    ]
 [0.     0.     3.0468]]
[3.60000000e+00 1.87390931e-03 2.55855664e+00 1.12969312e-03
 2.96366753e-03 3.04067755e+00]
[ 1.42140000e+00  1.87390931e-03 -3.74336035e-03  1.12969312e-03
  2.96366753e-03 -6.12245413e-03]


In [ ]:
positions = min_state.getPositions(asNumpy=True)
positions = positions.value_in_unit(unit.nano*unit.meter)
forces = min_state.getForces(asNumpy=True).value_in_unit(unit.kilojoule_per_mole/(unit.nano*unit.meter))
print(forces)
frac_positions = np.matmul(A_inv,positions.T).T
print(positions)


In [ ]:
print(box_vectors)
box_vector_array = np.array(box_vectors.value_in_unit(unit.nano*unit.meter)).flatten()
print(box_vector_array)
box_vector_array = np.delete(box_vector_array,[1,2,5]) #a_x,b_x,b_y,c_x,c_y,c_z
print(box_vector_array)
print(len(positions)/3)
